In [22]:
import numpy as np
import pandas as pd

import os
import glob

import matplotlib.pyplot as plt

In [2]:
experiment_name = 'RedGreen'
result_dir = '/home/arash/Desktop/Keras' + experiment_name + '/'

num_samples = 50
num_classes = 1000
num_images = num_samples * num_classes
class_inds = np.zeros((num_classes, 2))
for j, i in enumerate(range(0, num_images, num_samples)):
    class_inds[j, :] = [i, i + num_samples]

class ResultSummary():
    name = None
    top1_accuracy = None
    top5_accuracy = None
    classes_top1_accuracy = None
    classes_top5_accuracy = None
    test_index = 0
    
    def __init__(self, name, num_tests):
        self.name = name
        self.top1_accuracy = np.zeros((1, num_tests))
        self.top5_accuracy = np.zeros((1, num_tests))
        self.classes_top1_accuracy = np.zeros((num_classes, num_tests))
        self.classes_top5_accuracy = np.zeros((num_classes, num_tests))

    def add_test(self, predictions):
        self.top1_accuracy[0, self.test_index] = predictions[:, 0].mean()
        self.top5_accuracy[0, self.test_index] = predictions[:, 1].mean()
        for i in range(num_classes):
            si = int(class_inds[i, 0])
            ei = int(class_inds[i, 1])
            self.classes_top1_accuracy[i, self.test_index] = predictions[si:ei, 0].mean()
            self.classes_top5_accuracy[i, self.test_index] = predictions[si:ei, 1].mean()
        self.test_index += 1

In [3]:
num_networks = 13
num_tests = int(len(glob.glob(result_dir + '*.csv')) / num_networks)

networks = {
    'vgg16': ResultSummary('vgg16', num_tests),
    'vgg19': ResultSummary('vgg19', num_tests),
    'densenet121': ResultSummary('densenet121', num_tests),
    'densenet169': ResultSummary('densenet169', num_tests),
    'densenet201': ResultSummary('densenet201', num_tests),
    'mobilenet': ResultSummary('mobilenet', num_tests),
    'mobilenet_v2': ResultSummary('mobilenet_v2', num_tests),
    'nasnetmobile': ResultSummary('nasnetmobile', num_tests),
    'nasnetlarge': ResultSummary('nasnetlarge', num_tests),
    'resnet50': ResultSummary('resnet50', num_tests),
    'inception_v3': ResultSummary('inception_v3', num_tests),
    'inception_resnet_v2': ResultSummary('inception_resnet_v2', num_tests),
    'xception': ResultSummary('xception', num_tests)
}

for file in sorted(glob.glob(result_dir + '*.csv')):
    network_name = None
    if 'mobilenet_v2' in file:
        network_name = 'mobilenet_v2'
    else:
        for key in networks.keys():
            if key in file:
                network_name = key
                break
    if network_name is None:
        print(file)
    else:
        predictions = np.loadtxt(file, delimiter=',')
        networks[network_name].add_test(predictions)

In [4]:
top1_results = np.zeros((num_tests, num_networks))
top5_results = np.zeros((num_tests, num_networks))

for i, (key, value) in enumerate(networks.items()):
    top1_results[:, i] = value.top1_accuracy
    top5_results[:, i] = value.top5_accuracy

In [26]:
np.savetxt(experiment_name + '1.csv', top1_results, delimiter=',')
np.savetxt(experiment_name + '5.csv', top5_results, delimiter=',')

In [33]:
mean_changes = np.zeros((1, num_networks))
std_changes = np.zeros((1, num_networks))
outliers_n1 = []
outliers_p1 = []
for i, (key, value) in enumerate(networks.items()):
    average_change = value.top1_accuracy[0, 0] / value.top1_accuracy[0, num_tests-1]
    classes_change = value.classes_top1_accuracy[:, 0] / value.classes_top1_accuracy[:, num_tests-1]
    mean_changes[0, i] = classes_change.mean()
    std_changes[0, i] = classes_change.std()
    outliers_n1.append(np.where(classes_change < mean_change - std_change)[0])
    outliers_p1.append(np.where(classes_change > mean_change + std_change)[0])

In [32]:
for i, (key, value) in enumerate(networks.items()):
    print(key, len(outliers[i]))

vgg16 238
vgg19 206
densenet121 227
densenet169 206
densenet201 229
mobilenet 64
mobilenet_v2 234
nasnetmobile 163
nasnetlarge 57
resnet50 260
inception_v3 30
inception_resnet_v2 13
xception 32


In [34]:
set.intersection(*map(set,outliers_n1))

{1, 31, 46, 55, 211, 221, 365, 381, 738, 924, 943, 949, 950, 957}

In [41]:
outliers_n1

[array([  1,   3,   4,   6,   7,   8,  10,  12,  14,  15,  16,  27,  28,
         30,  31,  32,  33,  35,  37,  38,  39,  41,  45,  46,  47,  52,
         55,  56,  59,  60,  63,  64,  65,  67,  70,  78,  79,  84,  86,
         88,  90,  91,  94,  95,  96,  97,  99, 100, 104, 106, 113, 115,
        118, 119, 121, 122, 123, 125, 127, 128, 129, 130, 134, 136, 141,
        143, 149, 151, 156, 159, 161, 165, 166, 168, 171, 172, 173, 182,
        184, 185, 186, 187, 189, 191, 193, 201, 202, 204, 207, 209, 210,
        211, 213, 214, 215, 216, 218, 220, 221, 226, 227, 230, 231, 234,
        235, 236, 237, 238, 242, 246, 253, 258, 262, 263, 264, 265, 271,
        273, 274, 277, 282, 285, 286, 290, 294, 295, 298, 300, 301, 304,
        313, 314, 315, 316, 318, 320, 321, 322, 323, 329, 330, 333, 334,
        335, 337, 338, 339, 344, 345, 346, 351, 352, 353, 357, 358, 359,
        364, 365, 367, 371, 373, 374, 376, 377, 378, 379, 380, 381, 386,
        387, 390, 391, 393, 395, 407, 410, 411, 413

In [ ]:
nrows = 2
ncols = 1
ind = np.arange(num_classes)
for i, (key, value) in enumerate(networks.items()):
    plt.figure(num=key, figsize=(16, 4))
    j = 1
    plt.subplot(nrows, ncols, j)
    for k in range(num_tests):
        plt.bar(ind, value.classes_top1_accuracy[:, k])
    j += 1
    plt.subplot(nrows, ncols, j)
    for k in range(num_tests):
        plt.bar(ind, value.classes_top5_accuracy[:, k])
    j += 1